In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import scipy
import timeit
import itertools as it
%matplotlib inline

In [4]:
def algoB(B) :

    allval, allvect = scipy.linalg.eigh(B)
    valpos          = allval[allval>0]
    vectpos         = allvect[:, allval>0]
    S               = np.fliplr(np.array(vectpos>0, dtype=int)) ###fliplr permet d'organiser les vecteur propres du "grand" au plus "petit"
    pos_one         = (S[:,0]==1) #vecteur de positions des uns

    for i in range(1, len(valpos)):
        S[pos_one,i] = np.zeros(np.sum(pos_one>0)) #met des 0 aux positions où dans la colonne d'avant il y avait des uns.
        pos_one      = pos_one + (S[:,i]==1) #actualise le vecteur de positions des uns avec les uns de la colonne en cours
        if (np.sum(pos_one) >= n) :
            maxgroup = i+1
            break
    S_opt = S[:, :(maxgroup)]
    Q = np.trace(np.dot(np.dot(S_opt.transpose(),B),S_opt))/(2*m)
    return(dict(S_opt=S_opt, Q=Q, nb_group= maxgroup))

In [5]:
def algoC(x, B, xold, c=2, p=2):
    #initialisation des valeurs
    ng                = np.sum(x)

    #Calculs des termes diagonaux
    B_crop            = B[x==1,:][:,x==1]
    Bsum              = B_crop.sum(axis=1)
    B_g               = B_crop - np.diag(np.fromiter(iter(Bsum),dtype=float)) 
    
    #Diagonalisation de la matrice B_{ij}^G
    val_p_g, vect_p_g = scipy.linalg.eigh(B_g, eigvals=(len(B_g)-p,len(B_g)-1))
    
    #Calcul du alpha optimal
    allval, allvect   = scipy.linalg.eigh(B_g)
    alpha_g           = (1/(ng-p))*np.sum(allval[:(len(allval)-p)])
    
    #Calcul de la matrice des r_i
    Z_g               = np.tile(np.sqrt(val_p_g-alpha_g),ng).reshape(p,ng)
    r_mat_g           = Z_g*vect_p_g.transpose()
    
    #Algorithme
    count   = 0
    xg      = np.zeros(ng,dtype=int)
    xg[0]   = 1
    R_g     = r_mat_g[:,0]
    #vecteur permettant d'ajouter ou de retirer des sommets
    add   = np.zeros(n, dtype=int)  
    drop  = np.zeros(n, dtype=int)
    while count < ng:
        count = 0
        for i in range(0,ng):
            
            if np.dot(R_g, r_mat_g[:, i])>0:
                if (xg[i] != 1) :
                    add[i] = 1
                    xg[i]  = 1
                else:
                    count = count+1
            else:
            
                if (xg[i] ==1 ) :
                    drop[i] = 1
                    xg[i]   = 0
                else:
                    count = count+1
    R_g= R_g+ r_mat_g[:,add].sum(axis=1) - r_mat_g[:,drop].sum(axis=1) 
    
    #Stockages des valeurs dans une matrice S
    Sg       = np.zeros(ng*c, dtype=int).reshape(ng,c)
    Sg[:,0]  = xg
    Sg[:,1]  = 1-xg
    
    S = np.zeros(n*2,dtype=int).reshape(n,2)
    S[xold ==1,0]  = xg
    S[xold ==1,1]  = 1-xg

    
    #Calcul de la différence de modularité
    Delta_Q  = np.trace(np.dot(np.dot(Sg.transpose(), B_g), Sg))/(2*m)
    
    return(dict(Sg1=Sg[:,0], Sg2=Sg[:,1], Bg= B_g, dQ= Delta_Q, S1=S[:,0], S2=S[:,1]))

In [6]:
G  = nx.read_edgelist("facebook.txt", create_using = nx.Graph(), nodetype = int)
#G   = nx.read_gml('karate.gml')
#G   = nx.read_gml('lesmis.gml')
m   = G.number_of_edges()
n   = len(G)
A   = nx.adjacency_matrix(G)
deg = G.degree()
k   = np.fromiter(iter(deg.values()), dtype=int)
P   = np.outer(k,k)/(2*m)
B   = A-P

### ALGO A

In [302]:
# Leading eigenvector algorithm
val, vect = scipy.linalg.eigh(B, eigvals=(len(B)-1,len(B)-1))
valuesA         = np.array(vect.flatten()>0,dtype=int) -1*np.array(vect.flatten()<=0,dtype=int)

In [303]:
(1/(4*m))*np.dot(np.dot(valuesA,B),valuesA)

matrix([[ 0.36108097]])

### ALGO B 

In [304]:
val_algoB= algoB(B)

In [305]:
val_algoB["Q"]

0.51364777729555466

In [249]:
valuesB = np.zeros(n)
for i in range(val_algoB['S_opt'].shape[1]):
    valuesB = valuesB + (i+1)*val_algoB['S_opt'][:,i]

### ALGO C

In [7]:
x0 = np.ones(n,dtype=int)
x1 = algoC(x0,B,xold=x0)

In [8]:
x1["dQ"]

0.44592259926832223

In [9]:
sub1 = algoC(x1["Sg1"],x1["Bg"],xold=x1["S1"])
sub2 = algoC(x1["Sg2"],x1["Bg"],xold=x1["S2"])

In [11]:
print(sub1["dQ"], sub2["dQ"])

0.160229098053 0.0710668589417


In [12]:
sub11 = algoC(sub1["Sg1"],sub1["Bg"],xold=sub1["S1"])
sub12 = algoC(sub1["Sg2"],sub1["Bg"],xold=sub1["S2"])

In [13]:
print(sub11["dQ"],sub12["dQ"])

0.0683621890966 -0.0299957783337


In [14]:
sub21 = algoC(sub2["Sg1"],sub2["Bg"],xold=sub2["S1"])
sub22 = algoC(sub2["Sg2"],sub2["Bg"],xold=sub2["S2"])

In [15]:
print(sub21["dQ"],sub22["dQ"])

-4.05792467123e-19 5.11223884683e-07


In [16]:
sub111 = algoC(sub11["Sg1"],sub11["Bg"],xold=sub11["S1"])
sub112 = algoC(sub11["Sg2"],sub11["Bg"],xold=sub11["S2"])

In [17]:
print(sub111["dQ"], sub112["dQ"])

0.00403837614816 0.0093667284698


In [18]:
Sopt = np.zeros(n*100).reshape(n,100)

In [19]:
Sopt[:,1]= sub11["S1"]
Sopt[:,2]= sub11["S2"]
Sopt[:,3]= sub1["S2"]
Sopt[:,4]= sub2["S1"]
Sopt[:,5]= sub2["S2"]

In [20]:
np.trace(np.dot(np.dot(Sopt.transpose(),B),Sopt))/(2*m)

0.74558074535948116

In [21]:
valuesC = np.zeros(n)
for i in range(Sopt.shape[1]):
    valuesC = valuesC + (i+1)*Sopt[:,i]

### Représentation du graphe

In [ ]:
print(nx.info(G))

#Create network layout for visualizations
spring_pos = nx.spring_layout(G)

###
plt.axis("off")


In [ ]:
nx.draw_networkx(G, pos = spring_pos,  node_color = valuesC, 
                 font_size=20,node_size = 20, with_labels = False, cmap=plt.cm.Set1)